In [147]:
#importing libraries
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
%matplotlib inline

dtype = torch.FloatTensor

#function to impute the age feature
def impute_age(cols):
    Age=cols[0]
    Pclass=cols[1]
    if pd.isnull(Age) :
        if Pclass==1:
            return 37
        elif Pclass==2:
            return 29
        else:
            return 24
    else:
        return Age

#cleaning of the dataset
train=pd.read_csv('train.csv')
train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)
train.drop('Cabin',axis=1,inplace=True)
sex=pd.get_dummies(train['Sex'],drop_first=True)
embarked=pd.get_dummies(train['Embarked'],drop_first=True)
pclass=pd.get_dummies(train['Pclass'],drop_first=True)
train=pd.concat([train,sex,embarked,pclass],axis=1)
train.drop(['Sex','Embarked','Name','Ticket','Pclass'],axis=1,inplace=True)

#defining features and output matrices 
X=train[['Age', 'SibSp', 'Parch', 'Fare', 'male', 'Q', 'S', 2, 3]]
Y=train[['Survived']]
X=preprocessing.scale(X)
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=121)
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
Y_train=Y_train.as_matrix()
Y_test=Y_test.as_matrix()
Y_train=torch.FloatTensor(Y_train)
Y_test=torch.FloatTensor(Y_test)

#class for forward propagation
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 6)
        self.fc2 = nn.Linear(6, 1)    #1 hidden layer
#         self.fc3 = nn.Linear(6, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))       #using relu activation function in the hidden layer
        x = F.sigmoid(self.fc2(x))    #sigmoid function for final output layer
#         x = F.sigmoid(self.fc3(x))
        return x

net=Net()    #object of the class
# print(net)

criterion = nn.BCELoss()               #loss function
optimizer = optim.SGD(net.parameters(), lr=0.5)  #Using Stocastic Gradient Descent method for updating parameters
i=0



In [148]:
for epoch in range(12000):  # loop over the dataset multiple times
    running_loss = 0.0
    
    # get the inputs
    inputs=X_train
    labels=Y_train
#     print(inputs)
#     print(y_train)
    
    # wrap them in Variable
    x=Variable(inputs)
    y=Variable(labels)
    # zero the parameter gradients
    optimizer.zero_grad()
    
    # forward + backward + optimize
    outputs = net(x)
#     print(outputs)
#     print(y)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
#     print(loss)

In [149]:
X_t=Variable(X_test)
pred=net(X_t)
pred.shape
pred.data.numpy()     
for i in range(179):    #predicting for the test split
    if pred.data.numpy()[i][0]>=0.5:
        pred.data.numpy()[i][0]=1
    else:
        pred.data.numpy()[i][0]=0
# print(pred)

In [150]:
ac=(Y_test==pred.data).sum()/179   #checking the accuracy of the model
print(ac)

0.8212290502793296
